# Fix pathing

In [1]:
import sys


sys.path.append("..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))


# Imports

In [3]:
import datahandler
import plotter
import utils
import pathing

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import geopandas as gpd
import pandas as pd

import osmnx as ox
import networkx as nx
import geopandas as gpd
import shapely
import shapely.geometry
import folium
import scipy
import networkx


# Constants

In [4]:
data_preprocessor = datahandler.DataPreprocessorOUS_V2()
data_preprocessor.execute()

data_loader = datahandler.DataLoader(datahandler.DataPreprocessorOUS_V2)
data_loader.execute(clean=False, processed=False, enhanced=False)

Enhancing dataset: 100%|██████████| 2/2 [00:00<?, ?it/s]


# Main

In [5]:
od = pathing.OriginDestination(
    dataset_id="oslo",
    utm_epsg=f"EPSG:326{33}"
)

od.get_graph()

In [6]:
central_depot_grid_id = 22620006649000
central_depot_x, central_depot_y = utils.id_to_utm(central_depot_grid_id)
od.node_validator = ox.distance.nearest_nodes(od.graph, central_depot_x, central_depot_y)

In [7]:
def get_node(x, y):
    while True:
        node = ox.distance.nearest_nodes(od.graph, x, y)
        if networkx.has_path(od.graph, node, od.node_validator) and networkx.has_path(od.graph, od.node_validator, node):
            od.node_cache[(x, y)] = node
            return node
        else:
            od.graph.remove_node(node)

In [48]:
def evaluate_travel_time_accuracy(benchmark_times, calculated_times,verbose=False):
    total_diff = 0
    total_diff_percentage = 0

    for destination, benchmark in benchmark_times.items():
        if destination in calculated_times:
            calculated = calculated_times[destination]
            diff = abs(benchmark - calculated)
            diff_percentage = (diff / benchmark) * 100

            total_diff += diff
            total_diff_percentage += diff_percentage
            if verbose:
                print(f"Destination {destination}:")
                print(f"  Benchmark Time: {benchmark} minutes")
                print(f"  Calculated Time: {calculated} minutes")
                print(f"  Difference: {diff} minutes ({diff_percentage:.2f}%)")
                print()
        else:
            if verbose:
                print(f"Destination {destination} not found in calculated times.")

    avg_diff_percentage = total_diff_percentage / len(benchmark_times)
    if verbose:
        print(f"Total Discrepancy: {total_diff} minutes across all destinations.")
        print(f"Average Discrepancy Percentage: {avg_diff_percentage:.2f}%")
    return total_diff, avg_diff_percentage

In [49]:
intersection_penalty = 10
secret_scary_factor = 1 # 0.65
use_ambulance_speeds = False

od.set_graph_weights(intersection_penalty, secret_scary_factor, use_ambulance_speeds)

In [57]:
x, y = utils.geographic_to_utm(10.7343661,59.9369806)
start_node = get_node(x, y)

destinations = [(10.7740711,59.9276535), (10.7367191,59.9086137), (10.8131627,59.8967944)]

benchmark_times = {(10.7740711,59.9276535):7, # 6-8 minutes
                   (10.7367191,59.9086137):16, # 14-18 minutes
                   (10.8131627,59.8967944):14} # 14 minutter

In [58]:
# intersection_penalty = 15 # 10
# secret_scary_factor = 0.8 # 0.65
#intersection_penalty = 10 # 10
#secret_scary_factor = 0.65 # 0.65
intersection_penalty = 2
secret_scary_factor = 0.75
use_ambulance_speeds = False

od.set_graph_weights(intersection_penalty, secret_scary_factor, use_ambulance_speeds)

calculated_times = {}

for destination in destinations:
    x, y = utils.geographic_to_utm(destination[0], destination[1])
    destination_node = get_node(x, y)

    shortest_time_path = ox.shortest_path(
                    od.graph,
                    start_node,
                    destination_node,
                    weight='time'
                )

    total_travel_time = sum(od.graph[u][v][0]['time'] for u, v in zip(shortest_time_path[:-1], shortest_time_path[1:]))
    
    calculated_times[destination] = total_travel_time
    
evaluate_travel_time_accuracy(benchmark_times, calculated_times, verbose=True)

Destination (10.7740711, 59.9276535):
  Benchmark Time: 7 minutes
  Calculated Time: 5.549890533333333 minutes
  Difference: 1.4501094666666674 minutes (20.72%)

Destination (10.7367191, 59.9086137):
  Benchmark Time: 16 minutes
  Calculated Time: 8.781068666666668 minutes
  Difference: 7.218931333333332 minutes (45.12%)

Destination (10.8131627, 59.8967944):
  Benchmark Time: 14 minutes
  Calculated Time: 12.022520152380952 minutes
  Difference: 1.9774798476190476 minutes (14.12%)

Total Discrepancy: 10.646520647619047 minutes across all destinations.
Average Discrepancy Percentage: 26.65%


(10.646520647619047, 26.653008803854878)

In [53]:

locations = [
    (11.14674,60.31262),
    (11.47577,60.11964),
    (11.166858,60.13423),
    (11.561687,59.879044),
    (10.987775,59.932076),
    (10.880784,60.042152),
    (10.831094,59.930733),
    (10.758744,59.917),
    (10.741926,59.93917),
    (10.853331,59.71566),
    (10.725881,59.659687),
    (10.806946,59.833073),
    (10.439887,59.830055),
    (10.509723,59.89813),
    (10.687367,59.939663),
    (10.806364,59.893173),
    (10.884091,59.960304),
    (11.03928,59.997875),
    (10.585751,59.917873)
]

times = [
    40, 
    24,
    55,
    45,
    22,
    20,
    11.5, # 9-12
    11, # 8-14
    35,
    18,
    24,
    35,
    16,
    18,
    14, # 12-16
    14, # 12-16
    20, # 18-22
    35,
    60 # 55-65
]

benchmark_times = {(11.47577, 60.11964): 40,
                   (11.166858, 60.13423): 24,
                   (11.561687, 59.879044): 55,
                   (10.987775, 59.932076): 45,
                   (10.880784, 60.042152): 22,
                   (10.831094, 59.930733): 20,
                   (10.758744, 59.917): 11.5,
                   (10.741926, 59.93917): 11,
                   (10.853331, 59.71566): 35,
                   (10.725881, 59.659687): 18,
                   (10.806946, 59.833073): 24,
                   (10.439887, 59.830055): 35,
                   (10.509723, 59.89813): 16,
                   (10.687367, 59.939663): 18,
                   (10.806364, 59.893173): 14,
                   (10.884091, 59.960304): 14,
                   (11.03928, 59.997875): 20,
                   (10.585751, 59.917873): 35,
                   (11.14674, 60.31262): 60}

In [47]:
def get_node(x, y):
    if (x, y) in od.node_cache:
        return od.node_cache[(x, y)]

    while True:
        node = ox.distance.nearest_nodes(od.graph, x, y)
        if networkx.has_path(od.graph, node, od.node_validator) and networkx.has_path(od.graph, od.node_validator, node):
            od.node_cache[(x, y)] = node
            return node
        else:
            od.graph.remove_node(node)

In [55]:
from tqdm import tqdm  # Import tqdm

def find_optimal_parameters(locations, benchmark_times):
    optimal_diff = float('inf')
    optimal_settings = (None, None)

    # Wrap the outer loop with tqdm for a progress bar
    for intersection_penalty in tqdm([x * 0.5 for x in range(1, 31)], desc='Intersection Penalty Progress'):  # From 8 to 11 inclusive
        # Wrap the inner loop with tqdm as well, but you might want to remove or keep it silent to avoid clutter
        for secret_scary_factor in tqdm([x * 0.05 for x in range(1, 41)], desc='Secret Scary Factor Progress', leave=True):  # From 1.0 to 2.0 in 0.1 steps
            print(f"Current Settings: Intersection Penalty = {intersection_penalty}, Secret Scary Factor = {secret_scary_factor}")  # Print current parameter combination
            
            # Assuming a function to set simulation parameters exists
            calculated_times = calculate_times_for_all_locations(locations, intersection_penalty, secret_scary_factor)  # Pass current parameters to the function

            # Assuming a function to calculate times for all locations exists
            total_diff, avg_diff_percentage = evaluate_travel_time_accuracy(benchmark_times, calculated_times)

            if total_diff < optimal_diff:
                optimal_diff = total_diff
                optimal_settings = (intersection_penalty, secret_scary_factor)

    print(f"Optimal settings found: Intersection Penalty = {optimal_settings[0]}, Secret Scary Factor = {optimal_settings[1]}")
    print(f"With a total discrepancy of: {optimal_diff} minutes")

def calculate_times_for_all_locations(locations, intersection_penalty, secret_scary_factor):
    calculated_times = {}

    # It seems there's a mistake here; you should set the weights outside this function or pass them differently.
    # od.set_graph_weights(intersection_penalty, secret_scary_factor, use_ambulance_speeds)

    for end_point, _ in benchmark_times.items():
        start_index = (locations.index(end_point) - 1) % len(locations)  # Ensuring the index wraps correctly
        start_point = locations[start_index]
        
        start_x, start_y = utils.geographic_to_utm(start_point[0], start_point[1])
        end_x, end_y = utils.geographic_to_utm(end_point[0], end_point[1])
        
        start_node = get_node(start_x, start_y)
        end_node = get_node(end_x, end_y)

        od.set_graph_weights(intersection_penalty, secret_scary_factor, use_ambulance_speeds=False)

        shortest_time_path = ox.shortest_path(
            od.graph,
            start_node,
            end_node, 
            weight='time')
        total_travel_time = sum(od.graph[u][v][0]['time'] for u, v in zip(shortest_time_path[:-1], shortest_time_path[1:]))

        calculated_times[end_point] = total_travel_time
    return calculated_times

# You would call find_optimal_parameters like this, with your locations and benchmark_times
find_optimal_parameters(locations, benchmark_times)


Intersection Penalty Progress:   0%|          | 0/17 [00:00<?, ?it/s]

Current Settings: Intersection Penalty = 2.0, Secret Scary Factor = 0.25


Current Settings: Intersection Penalty = 2.0, Secret Scary Factor = 0.30000000000000004


Current Settings: Intersection Penalty = 2.0, Secret Scary Factor = 0.35000000000000003


Current Settings: Intersection Penalty = 2.0, Secret Scary Factor = 0.4


...

Current Settings: Intersection Penalty = 10.0, Secret Scary Factor = 0.65


Current Settings: Intersection Penalty = 10.0, Secret Scary Factor = 0.7000000000000001


Current Settings: Intersection Penalty = 10.0, Secret Scary Factor = 0.75


Intersection Penalty Progress: 100%|██████████| 17/17 [23:37<00:00, 83.39s/it]

Optimal settings found: Intersection Penalty = 2.0, Secret Scary Factor = 0.75
With a total discrepancy of: 38.2072466631744 minutes
